In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import numpy as np
import  pandas as pd

from sklearn.naive_bayes import GaussianNB,ComplementNB,MultinomialNB
from sklearn.mixture import BayesianGaussianMixture

from sklearn.metrics import accuracy_score,log_loss
from sklearn.model_selection import train_test_split
from pandas_ml import ConfusionMatrix as cm
from sklearn.preprocessing import StandardScaler
from sklearn import svm

from sklearn.neighbors.kde import KernelDensity

from IPython.display import display_html

from scipy.stats import norm

import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
import statsmodels as sm
import matplotlib

from IPython.display import display,clear_output,HTML

In [19]:
#### Source= https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

def mydisplay(dfs, names=[]):
    html_str = ''
    if names:
        html_str += ('<tr>' + 
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
                     '</tr>')
    html_str += ('<tr>' + 
                 ''.join(f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>' 
                         for df in dfs) + 
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

In [20]:
train=pd.read_csv('Feature_Vector_50_50_12_24_random_state_0.csv',index_col='User')
print('Training Set\nInfected/Unifected Users')
print()
display(train.Target.value_counts())
user_Dec=train.index.values

jan=pd.read_csv('Feature-Set_Jan_Final.csv',index_col='User')
print('\nOriginal Jan data\nInfected/Unifected Users')
display(jan.Target.value_counts())
user_jan=jan.index.values

no=[user for user in user_jan if user not in user_Dec]

o=[user for user in user_jan if user in user_Dec]

jan.drop(o,inplace=True)

print("\nAfter eliminating overlapping users:")
print(jan.Target.value_counts())

# x=jan[jan.Target==1]
# y=jan[jan.Target==0]
# y=y.sample(n=532,random_state=10)
# z=x.append(y)
z=jan.sample(frac=1,random_state=10)
Y_test=z.Target.values

Training Set
Infected/Unifected Users



1    2910
0    2910
Name: Target, dtype: int64


Original Jan data
Infected/Unifected Users


0    6406
1     532
Name: Target, dtype: int64


After eliminating overlapping users:
0    6181
1     532
Name: Target, dtype: int64


## For January - including the Infected Users from March as well

### Individual

In [21]:
Y_test=z.Target.values

for col in train.columns[:-1]:
    train[col]=np.log(train[col])
train.replace([np.inf,-np.inf],0,inplace=True)

for col in z.columns[:-1]:
    z[col]=np.log(z[col])
z.replace([np.inf,-np.inf],0,inplace=True)

In [5]:
lst=[]
for col in train.columns[:-1]:
    
    ### Non-Malicious Users
    X_train=train[train.Target==0][col].values
    Y_train=train[train.Target==0]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = GaussianNB()
    clf.fit(scaled_X,Y_train)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    non_mal_lst=clf.predict(X_test)
    mean=clf.theta_[0][0]
    std=np.sqrt(clf.sigma_[0][0])
    non_mal_lst=[]
    for x in X_test:
        non_mal_lst.append((norm.pdf(x[0],mean,std)))
    
    
    ### Malicious users
    X_train=train[train.Target==1][col].values
    Y_train=train[train.Target==1]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = GaussianNB()
    clf.fit(scaled_X,Y_train)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    mal_lst=clf.predict(X_test)
    mean=clf.theta_[0][0]
    std=np.sqrt(clf.sigma_[0][0])
    mal_lst=[]
    for x in X_test:
        mal_lst.append((norm.pdf(x[0],mean,std)))
    
    
    df=pd.DataFrame({'Non_Mal':non_mal_lst,'Mal':mal_lst,
              'Ground_Truth':Y_test.reshape(1,-1)[0].tolist()},index=z.index.values)


    df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
    df['Predicted']=np.where(df['Non_Mal'] <= df['Mal'], 'Malicious','Non_Malicious')
    print('\033[1m' + str(col) + '\033[0m\n')
    display(pd.crosstab(df.Predicted,df.Ground_Truth))
    print("-----------------------------------------------------------------------------------------------------")
    file_name='Parametric/Jan/Para_col_'+str(col)+'.csv'
    df.to_csv(file_name)

SP



Ground_Truth,Malicious,Non_Malicious
Predicted,,
Malicious,249,583
Non_Malicious,283,5598


-----------------------------------------------------------------------------------------------------
DP



Ground_Truth,Malicious,Non_Malicious
Predicted,,
Malicious,177,605
Non_Malicious,355,5576


-----------------------------------------------------------------------------------------------------
NAPP



...

-----------------------------------------------------------------------------------------------------
2200



Ground_Truth,Malicious,Non_Malicious
Predicted,,
Malicious,339,1273
Non_Malicious,193,4908


-----------------------------------------------------------------------------------------------------
2300



Ground_Truth,Malicious,Non_Malicious
Predicted,,
Malicious,337,1302
Non_Malicious,195,4879


-----------------------------------------------------------------------------------------------------


### Combined

In [30]:
col=train.columns[:-1]

In [31]:
X_train=train[col].values
Y_train=train['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=100)
clf.fit(scaled_X,Y_train)

X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.to_csv('Parametric/Jan/Para_Combined.csv')

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,1.0,1.031127e-50,Non_Malicious,Malicious
1,1.0,7.382480e-261,Non_Malicious,Malicious
2,1.0,2.219460e-273,Non_Malicious,Malicious
3,1.0,5.249962e-55,Non_Malicious,Malicious
4,1.0,0.000000e+00,Non_Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,434,98
Non_Malicious,514,18


## For January - not including the Infected Users from March

### Combined

In [32]:
clf.predict(X_test)

x=z[z.Target==0]
y=z[z.Target==1]
x=x.sample(n=532,random_state=10)

z=x.append(y)
display(z.Target.value_counts())

col=train.columns[:-1]

1    532
0    532
Name: Target, dtype: int64

#### For all users

In [33]:
X_train=train[col].values
Y_train=train['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=1500)
clf.fit(scaled_X,Y_train)

Y_test=z.Target.values
X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.set_index(z.index.values,drop=True,inplace=True)
m.to_csv('Parametric/Jan/Combined_All_Users.csv',index=True)

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,1.0,2.365020e-14,Non_Malicious,Malicious
1,1.0,7.848163e-90,Non_Malicious,Malicious
2,1.0,1.680014e-92,Non_Malicious,Malicious
3,1.0,0.000000e+00,Non_Malicious,Malicious
4,1.0,8.917636e-23,Non_Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,434,98
Non_Malicious,514,18


#### For non- malicious Users

In [34]:
X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=1500)
clf.fit(scaled_X,Y_train)

Y_test=z.Target.values
X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.set_index(z.index.values,drop=True,inplace=True)
m.to_csv('Parametric/Jan/Combined_Non_Malicious_Users.csv',index=True)

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,1.000000e+00,0.000000e+00,Non_Malicious,Malicious
1,1.000000e+00,5.555140e-193,Non_Malicious,Malicious
2,2.185085e-08,1.000000e+00,Non_Malicious,Non_Malicious
3,1.000000e+00,0.000000e+00,Non_Malicious,Malicious
4,1.000000e+00,0.000000e+00,Non_Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,484,48
Non_Malicious,316,216


#### For malicious users

In [35]:
X_train=train[train.Target==1][col].values
Y_train=train[train.Target==1]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=1500)
clf.fit(scaled_X,Y_train)

Y_test=z.Target.values
X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.set_index(z.index.values,drop=True,inplace=True)
m.to_csv('Parametric/Jan/Combined_Malicious_Users.csv',index=True)

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,1.0,1.015543e-19,Non_Malicious,Malicious
1,1.0,1.643115e-103,Non_Malicious,Malicious
2,1.0,3.099703e-115,Non_Malicious,Malicious
3,1.0,0.000000e+00,Non_Malicious,Malicious
4,1.0,6.190358e-27,Non_Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,477,55
Non_Malicious,525,7


## For February data - inlcuding infected users from March as well

### Combined

In [36]:
train=pd.read_csv('Feature_Vector_50_50_12_24_random_state_0.csv',index_col='User')
print('Training Set\nInfected/Unifected Users')
print()
display(train.Target.value_counts())
user_Dec=train.index.values

Feb=pd.read_csv('Feature-Set_Feb_Final.csv',index_col='User')
print('\nOriginal Feb data\nInfected/Unifected Users')
display(Feb.Target.value_counts())
user_Feb=Feb.index.values

no=[user for user in user_Feb if user not in user_Dec]

o=[user for user in user_Feb if user in user_Dec]

Feb.drop(o,inplace=True)

print("\nAfter eliminating overlapping users:")
print(Feb.Target.value_counts())

# x=Feb[Feb.Target==1]
# y=Feb[Feb.Target==0]
# y=y.sample(n=532,random_state=10)
# z=x.append(y)
z=Feb.sample(frac=1,random_state=10)
Y_test=z.Target.values

Training Set
Infected/Unifected Users



1    2910
0    2910
Name: Target, dtype: int64


Original Feb data
Infected/Unifected Users


0    6125
1    1131
Name: Target, dtype: int64


After eliminating overlapping users:
0    5899
1    1131
Name: Target, dtype: int64


In [37]:
col=train.columns[:-1]

In [38]:
X_train=train[col].values
Y_train=train['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=100)
clf.fit(scaled_X,Y_train)

X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.to_csv('Parametric/Feb/Para_Combined.csv')

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,0.000000e+00,1.000000e+00,Non_Malicious,Non_Malicious
1,1.000000e+00,2.166521e-19,Non_Malicious,Malicious
2,1.045623e-16,1.000000e+00,Non_Malicious,Non_Malicious
3,3.497362e-86,1.000000e+00,Malicious,Non_Malicious
4,1.000000e+00,3.928117e-34,Non_Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,299,832
Non_Malicious,3865,2034


## For February - not including the Infected Users from March

### Combined

In [39]:
clf.predict(X_test)

x=z[z.Target==0]
y=z[z.Target==1]
x=x.sample(n=1131,random_state=10)

z=x.append(y)
display(z.Target.value_counts())

col=train.columns[:-1]

1    1131
0    1131
Name: Target, dtype: int64

#### For all users

In [40]:
X_train=train[col].values
Y_train=train['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=1500)
clf.fit(scaled_X,Y_train)

Y_test=z.Target.values
X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.set_index(z.index.values,drop=True,inplace=True)
m.to_csv('Parametric/Feb/Combined_All_Users.csv',index=True)

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,1.000000e+00,4.241567e-84,Non_Malicious,Malicious
1,1.000000e+00,3.284522e-90,Non_Malicious,Malicious
2,1.000000e+00,1.033206e-169,Non_Malicious,Malicious
3,4.204426e-30,1.000000e+00,Non_Malicious,Non_Malicious
4,1.000000e+00,4.919614e-76,Non_Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,832,299
Non_Malicious,373,758


#### For non- malicious Users

In [41]:
X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=1500)
clf.fit(scaled_X,Y_train)

Y_test=z.Target.values
X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.set_index(z.index.values,drop=True,inplace=True)
m.to_csv('Parametric/Feb/Combined_Non_Malicious_Users.csv',index=True)

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,1.000000e+00,1.554753e-274,Non_Malicious,Malicious
1,1.000000e+00,0.000000e+00,Non_Malicious,Malicious
2,1.000000e+00,0.000000e+00,Non_Malicious,Malicious
3,2.551225e-28,1.000000e+00,Non_Malicious,Non_Malicious
4,1.000000e+00,4.410728e-256,Non_Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,977,154
Non_Malicious,546,585


#### For malicious users

In [46]:
X_train=train[train.Target==1][col].values
Y_train=train[train.Target==1]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=1500)
clf.fit(scaled_X,Y_train)

Y_test=z.Target.values
X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.set_index(z.index.values,drop=True,inplace=True)
m.to_csv('Parametric/Feb/Combined_Malicious_Users.csv',index=True)

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,1.0,2.678864e-16,Non_Malicious,Malicious
1,1.0,4.396500e-16,Non_Malicious,Malicious
2,1.0,6.734975e-17,Non_Malicious,Malicious
3,1.0,2.137094e-18,Non_Malicious,Malicious
4,1.0,1.437722e-20,Non_Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,921,210
Non_Malicious,1075,56


## For March data - inlcuding infected users from March as well

### Combined

In [57]:
train=pd.read_csv('Feature_Vector_50_50_12_24_random_state_0.csv',index_col='User')
print('Training Set\nInfected/Unifected Users')
print()
display(train.Target.value_counts())
user_Dec=train.index.values

Mar=pd.read_csv('Feature-Set_Mar.csv',index_col='User')
print('\nOriginal Mar data\nInfected/Unifected Users')
display(Mar.Target.value_counts())
user_Mar=Mar.index.values

no=[user for user in user_Mar if user not in user_Dec]

o=[user for user in user_Mar if user in user_Dec]

Mar.drop(o,inplace=True)

print("\nAfter eliminating overlapping users:")
print(Mar.Target.value_counts())

x=Mar[Mar.Target==1]
y=Mar[Mar.Target==0]
y=y.sample(n=4787,random_state=10)
z=x.append(y)
z=z.sample(frac=1,random_state=10)
Y_test=z.Target.values
print("\nAfter eliminating overlapping users:")
print(z.Target.value_counts())

Training Set
Infected/Unifected Users



1    2910
0    2910
Name: Target, dtype: int64


Original Mar data
Infected/Unifected Users


0    54891
1     5931
Name: Target, dtype: int64


After eliminating overlapping users:
0    51035
1     4787
Name: Target, dtype: int64

After eliminating overlapping users:
1    4787
0    4787
Name: Target, dtype: int64


In [61]:
col=train.columns[:-1]

In [64]:
X_train=train[col].values
Y_train=train['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=100)
clf.fit(scaled_X,Y_train)

X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.to_csv('Parametric/Mar/Combined_All_Users.csv')

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,3.804081e-29,1.0,Non_Malicious,Non_Malicious
1,2.574856e-26,1.0,Non_Malicious,Non_Malicious
2,7.900074e-19,1.0,Non_Malicious,Non_Malicious
3,6.694796e-29,1.0,Non_Malicious,Non_Malicious
4,1.000000e+00,0.0,Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,3088,1699
Non_Malicious,1146,3641


#### For non- malicious Users

In [65]:
X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=1500)
clf.fit(scaled_X,Y_train)

Y_test=z.Target.values
X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.set_index(z.index.values,drop=True,inplace=True)
m.to_csv('Parametric/Mar/Combined_Non_Malicious_Users.csv',index=True)

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,2.952593e-27,1.0,Non_Malicious,Non_Malicious
1,4.764293e-16,1.0,Non_Malicious,Non_Malicious
2,1.706837e-10,1.0,Non_Malicious,Non_Malicious
3,4.292358e-27,1.0,Non_Malicious,Non_Malicious
4,1.000000e+00,0.0,Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,3544,1243
Non_Malicious,1598,3189


#### For malicious users

In [66]:
X_train=train[train.Target==1][col].values
Y_train=train[train.Target==1]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)    
clf = BayesianGaussianMixture(n_components=2,max_iter=1500)
clf.fit(scaled_X,Y_train)

Y_test=z.Target.values
X_test=scaler.transform(z[col])
lst=clf.predict_proba(X_test)

m=pd.DataFrame({'Malicious':lst[:,1],'Non_Malicious':lst[:,0],'Ground_Truth':Y_test})
m['Predicted']=np.where(m['Malicious']>m['Non_Malicious'],'Malicious','Non_Malicious')
m['Ground_Truth']=m.Ground_Truth.apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(m.head())

display(pd.crosstab(m.Ground_Truth,m.Predicted))

m.set_index(z.index.values,drop=True,inplace=True)
m.to_csv('Parametric/Mar/Combined_Malicious_Users.csv',index=True)

,Malicious,Non_Malicious,Ground_Truth,Predicted
0,1.0,1.520357e-18,Non_Malicious,Malicious
1,1.0,5.663305e-17,Non_Malicious,Malicious
2,1.0,4.075770e-18,Non_Malicious,Malicious
3,1.0,7.637323e-21,Non_Malicious,Malicious
4,1.0,2.301004e-11,Malicious,Malicious


Predicted,Malicious,Non_Malicious
Ground_Truth,,
Malicious,4076,711
Non_Malicious,4558,229
